#**0. Preparation of the environment**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Confirmamos la ruta donde está el archivo csv
%ls /content/drive/MyDrive/Colab_Notebooks/GITHUB/DOORDASH/historical_data.csv

In [ ]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np

# Cargamos el archivo csv en un DataFrame
delivery_data = pd.read_csv(
    "/content/drive/MyDrive/Colab_Notebooks/GITHUB/DOORDASH/historical_data.csv", encoding='utf-8', encoding_errors='replace')

#**1. Data description**


**Time features**

* market_id: A city/region in which DoorDash operates, e.g., Los Angeles, given in the data as an id
* created_at: Timestamp in UTC when the order was submitted by the consumer to DoorDash. (Note this timestamp is in UTC, but in case you need it, the actual timezone of the region was US/Pacific)
* actual_delivery_time: Timestamp in UTC when the order was delivered to the consumer

**Store features**

* store_id: an id representing the restaurant the order was submitted for
*store_primary_category: cuisine category of the restaurant, e.g., italian, asian
*order_protocol: a store can receive orders from DoorDash through many modes. This field represents an id denoting the protocol

**Order features**

* total_items: total number of items in the order
* subtotal: total value of the order submitted (in cents)
* num_distinct_items: number of distinct items included in the order
* min_item_price: price of the item with the least cost in the order (in cents)
* max_item_price: price of the item with the highest cost in the order (in cents)

**Market features**

DoorDash being a marketplace, we have information on the state of marketplace when the order is placed, that can be used to estimate delivery time. The following features are values at the time of created_at (order submission time):

*total_onshift_dashers: Number of available dashers who are within 10 miles of the store at the time of order creation
*total_busy_dashers: Subset of above total_onshift_dashers who are currently working on an order
*total_outstanding_orders: Number of orders within 10 miles of this order that are currently being processed.

**Predictions from other models**

We have predictions from other models for various stages of delivery process that we can use:

* estimated_order_place_duration: Estimated time for the restaurant to receive the order from DoorDash (in seconds)
* estimated_store_to_consumer_driving_duration: Estimated travel time between store and consumer (in seconds)

When a consumer places an order on DoorDash, we show the expected time of delivery. It is very important for DoorDash to get this right, as it has a big impact on consumer experience.

In this project we are going to predict the total delivery duration (in seconds) based on the moment the consumer submits de order (created_at) and when the order will be delivered to the consumer (actual_delivery_time)

#**2. Data Exploration**

In [ ]:
delivery_data.head()

In [ ]:
delivery_data.info()

In [ ]:
# Creating the target variable for regression
from datetime import datetime
delivery_data["created_at"] = pd.to_datetime(delivery_data['created_at'])
delivery_data["actual_delivery_time"] = pd.to_datetime(delivery_data['actual_delivery_time'])
delivery_data["actual_total_delivery_duration"] = (delivery_data["actual_delivery_time"] - delivery_data["created_at"]).dt.total_seconds()
delivery_data.head()

In [ ]:
# Creating new features
# We can merge the estimated time of the restaurante to receive an order and the estimated time that takes from the store to deliver to the consumer as the non preparation duration
delivery_data['estimated_non_prep_duration'] = delivery_data["estimated_store_to_consumer_driving_duration"] + delivery_data["estimated_order_place_duration"]
# Ratio of available dashers at the time of order creation
delivery_data["busy_dashers_ratio"] = delivery_data["total_busy_dashers"] / delivery_data["total_onshift_dashers"]
delivery_data.head()

##**2.1. Managing nulls**

In [ ]:
print("NaN per column\n: ", delivery_data.isna().sum())

In [ ]:
# Creating dictionary with most frequent category of each store to fill null
store_id_unique = delivery_data["store_id"].unique().tolist()
store_id_and_category = {store_id: delivery_data[delivery_data.store_id == store_id].store_primary_category.mode()
                         for store_id in store_id_unique}

In [ ]:
def fill(store_id):
    """Return mode store category from the dictionary"""
    try:
        return store_id_and_category[store_id].values[0]
    except:
        return np.nan

# fill null values
delivery_data["nan_free_store_primary_category"] = delivery_data.store_id.apply(fill)

##**2.2. Encoding**

In [ ]:
#Check if it makes sense to onehot encode nominal data columns
print(delivery_data['market_id'].nunique())
print(delivery_data['store_id'].nunique())
print(delivery_data['nan_free_store_primary_category'].nunique())
print(delivery_data['order_protocol'].nunique())

In [ ]:
# Dummies for market_id, order_protocol and nan_free_store_primary_category
delivery_data = pd.get_dummies(delivery_data,
                            columns=['market_id', 'order_protocol', 'nan_free_store_primary_category'],
                            prefix=['market_id', 'order_protocol', 'category']
                            )
print(delivery_data.columns)

In [ ]:
# drop unnecessary columns
train_df = delivery_data.drop(columns = ["created_at", "store_id", "store_primary_category", "actual_delivery_time"])
train_df.head()

In [ ]:
train_df.describe()

In [ ]:
# replace inf values with nan
train_df.replace([np.inf, -np.inf], np.nan, inplace=True)
# drop all nans
train_df.dropna(inplace=True)

In [ ]:
train_df.describe()

##**2.3. Collinearity**

In [ ]:
train_df.info()

In [ ]:
# align dtype over dataset
train_df = train_df.astype("float32")

In [ ]:
train_df.shape

We have 100 columns in our final dataset, which means there might be redundant features and potential collinearity which doesn't add any new knowledge to ML models. We start with the correlation matrix.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Generate a mask for the upper triangle because the other traingle is its summetry. This way we have a better visualization
correlation_matrix = train_df.corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))

# Drawing the heatmap with the mask
plt.figure(figsize = (11,9))
sns.heatmap(correlation_matrix, mask=mask, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5}, cmap="coolwarm")
plt.show()

In [ ]:
#In the correlation matrix looks like there is an issue in category_indonesian
train_df['category_indonesian'].describe()

As we can see, category_indonesian has a lot of zeros as value, so we can drop this feature beacuse it has no effect. In order to drop redundant values and find the top correlated features we are going to use two functions. This way we can avoid multicollinearity by removing highly correlated features

In [ ]:
def get_redundant_pairs(df):
    """Get diagonal and lower triangular pairs of correlation matrix"""
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_top_abs_correlations(df, n=5):
    """Sort correlations in the descending order and return n highest results"""
    au_corr = df.corr().abs().unstack()
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop).sort_values(ascending=False)
    return au_corr[0:n]

print("Top Absolute Correlations")
print(get_top_abs_correlations(train_df, 20))

In [ ]:
# drop highly correlated features
train_df = train_df.drop(columns=["total_onshift_dashers", "total_busy_dashers", "category_indonesian",
                                  "estimated_non_prep_duration", "market_id_1.0", "market_id_2.0",
                                  "market_id_3.0", "market_id_4.0", "market_id_5.0", "market_id_6.0"])

In [ ]:
print("Top Absolute Correlations")
print(get_top_abs_correlations(train_df, 20))

In [ ]:
train_df = train_df.drop(columns=["order_protocol_1.0", "order_protocol_2.0", "order_protocol_3.0", "order_protocol_4.0",
                                  "order_protocol_5.0", "order_protocol_6.0", "order_protocol_7.0"])

In [ ]:
print("Top Absolute Correlations")
print(get_top_abs_correlations(train_df, 20))

The total item number or distinct items could affect the duration of the preparation process. Therefore, we do not prefer to drop them. Instead, we can use the power of feature engineering. We will create new columns to infer the contribution of these columns.

In [ ]:
# new features
train_df["percent_distinct_item_of_total"] = train_df["num_distinct_items"] / train_df["total_items"]
train_df["avg_price_per_item"] = train_df["subtotal"] / train_df["total_items"]
train_df.drop(columns=["num_distinct_items", "subtotal"], inplace=True)
print("Top Absolute Correlations")
print(get_top_abs_correlations(train_df, 20))

In [ ]:
train_df["price_range_of_items"] = train_df["max_item_price"] - train_df["min_item_price"]
train_df.drop(columns=["max_item_price", "min_item_price"], inplace=True)
print("Top Absolute Correlations")
print(get_top_abs_correlations(train_df, 20))

In [ ]:
train_df.shape

The feature engineering has improved the dataset and now we have 82 features with less than 0.5 collinearity between them

##**2.4. Multicollinearity**


To avoid having multiple variables correlated with each other we are going to use VIF which tells us the score by measuring how much our regression analysis is affected by collinearity. We will remove the features that have a vif score of over 20. This way it will be easier to interpret the model and reduce the risk of overffiting

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def compute_vif(features):
    """Compute VIF score using variance_inflation_factor() function"""
    vif_data = pd.DataFrame()
    vif_data["feature"] = features
    vif_data["VIF"] = [variance_inflation_factor(train_df[features].values, i) for i in range(len(features))]
    return vif_data.sort_values(by=['VIF']).reset_index(drop=True)

In [ ]:
# apply VIF to all columns
features = train_df.drop(columns=["actual_total_delivery_duration"]).columns.to_list()
vif_data = compute_vif(features)
vif_data

In [ ]:
# Drop features with the highest VIF score until all VIF scores are below 20
while vif_data['VIF'].max() > 20:
    highest_vif_feature = vif_data.loc[vif_data['VIF'].idxmax(), 'feature']  # Get the feature with the highest VIF
    print(f"Removing {highest_vif_feature} due to high VIF")

    features.remove(highest_vif_feature)  # Remove the feature
    vif_data = compute_vif(features)  # Recalculate VIF

# Store the final selected features
selected_features = vif_data['feature'].tolist()

##**2.5. Feature selection**


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

X = train_df[selected_features]
y = train_df["actual_total_delivery_duration"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
feature_names = [f"feature {i}" for i in range((X.shape[1]))]
forest = RandomForestRegressor(random_state=42)
forest.fit(X_train, y_train)
feats = {} # a dict to hold feature_name: feature_importance
for feature, importance in zip(X.columns, forest.feature_importances_):
    feats[feature] = importance #add the name/value pair

importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
importances.sort_values(by='Gini-importance').plot(kind='bar', rot=90, figsize=(15,12))
plt.show()

In [ ]:
# check the most important ones
importances.sort_values(by='Gini-importance')[-35:].plot(kind='bar', rot=90, figsize=(15,12))
plt.show()

In [ ]:
# apply PCA to see feature contributions
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X_Train=X_train.values
X_Train=np.asarray(X_Train)

# Finding normalised array of X_Train
X_std=StandardScaler().fit_transform(X_Train)
pca = PCA().fit(X_std)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlim(0,81)
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')
plt.show()

PCA shows that we need to use at least 60 representative features to explain 80% of the dataset, which makes the PCA transformation useless since we already have 80 and could select the most important ones based on feature importance. However, if PCA would tell us it can explain the majority of variance with around 10 features - high reduction - we would continue with it

#**3.Machine Learning**

##**3.1. Classic machine learning regression**

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

def calculate_rmse(y_test, y_pred, model_name):
    """Calculate and print RMSE for a regression model"""

    # Compute RMSE (Root Mean Squared Error)
    rmse_error = mean_squared_error(y_test, y_pred)

    # Print the result
    print(f"Error (RMSE) = {rmse_error:.4f} in {model_name}")

    return rmse_error

We will use Root Mean Squared Error to measure error based on the sensitivity of RMSE for high error terms. In our thought, the consumer patience with delaying delivery could decrease exponentially with time.

In [ ]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

In [ ]:
# Initialize prediction dictionary to store RMSE results
pred_dict = {}

# Define models and settings
models = {
    "Linear Regression": LinearRegression(),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "SVM Regression": SVR(kernel="rbf"),
    "XGBoost": XGBRegressor(objective="reg:squarederror", n_estimators=100, random_state=42),
}

# Train and evaluate each model
for name, model in models.items():
    model.fit(X_train_scaled, y_train)
    y_pred = model.predict(X_test_scaled)
    pred_dict[name] = calculate_rmse(y_test, y_pred, name)

# Polynomial Regression with different degrees
poly_degrees = [2]

for degree in poly_degrees:
    poly = PolynomialFeatures(degree=degree)
    X_train_poly = poly.fit_transform(X_train_scaled)
    X_test_poly = poly.transform(X_test_scaled)

    poly_model = LinearRegression()
    poly_model.fit(X_train_poly, y_train)
    y_pred_poly = poly_model.predict(X_test_poly)

    model_name = f"Polynomial Regression (Degree {degree})"
    pred_dict[model_name] = calculate_rmse(y_test, y_pred_poly, model_name)

##**3.2. Deep Learning**